# Conversation Management & JSON Extraction with Groq API (OpenAI-compatible)

This Colab-ready notebook implements:

- Conversation management with history truncation and periodic summarization
- JSON Schema–style classification & information extraction via OpenAI-style tool/function calling on the Groq API

Resources:
- [Groq API (OpenAI-compatible) docs](https://console.groq.com/docs)
- [OpenAI Python SDK docs](https://platform.openai.com/docs/api-reference)

Run all cells top-to-bottom. Provide a valid Groq API key when prompted.


In [ ]:
# Setup: install and imports
import sys
import json
import os
import textwrap
from dataclasses import dataclass, field
from typing import List, Dict, Any, Optional, Callable

# In Colab, ensure openai client is installed
try:
    import openai  # type: ignore
except Exception:
    !pip -q install --upgrade openai
    import openai  # type: ignore

# Configure OpenAI-compatible client for Groq
# Groq exposes an OpenAI-compatible endpoint and key
# Load from .env (no external libs) and allow environment override

def load_env_file(path: str = ".env") -> None:
    try:
        if not os.path.exists(path):
            return
        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip()
                if not line or line.startswith("#"):
                    continue
                if "=" in line:
                    key, value = line.split("=", 1)
                    key = key.strip()
                    value = value.strip().strip('\'"')
                    if key and os.getenv(key) is None:
                        os.environ[key] = value
    except Exception:
        pass

load_env_file()

GROQ_API_KEY = os.getenv("GROQ_API_KEY", "").strip()
if not GROQ_API_KEY:
    try:
        from getpass import getpass
        GROQ_API_KEY = getpass("Enter your GROQ_API_KEY: ")
    except Exception:
        GROQ_API_KEY = input("Enter your GROQ_API_KEY: ")

# Keep env variable in sync for any downstream libraries
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

openai.api_key = GROQ_API_KEY
openai.base_url = "https://api.groq.com/openai/v1"

# Choose a Groq chat model compatible with OpenAI's Chat Completions
# Recommended examples (subject to change):
# - llama-3.1-70b-versatile
# - llama-3.1-8b-instant
# - mixtral-8x7b-32768
GROQ_MODEL = os.getenv("GROQ_MODEL") or "llama-3.1-70b-versatile"
MODEL_FALLBACKS = [
    "llama-3.1-70b-versatile",
    "llama-3.1-8b-instant",
    "mixtral-8x7b-32768",
]
print("Configured model:", GROQ_MODEL)



In [ ]:
# Create OpenAI-compatible client for Groq (v1 SDK style)
try:
    from openai import OpenAI
    client = OpenAI(base_url="https://api.groq.com/openai/v1", api_key=GROQ_API_KEY)
except Exception as e:
    print("Falling back to legacy openai module usage due to:", e)
    client = None



In [ ]:
# Conversation Manager with truncation and periodic summarization

@dataclass
class Message:
    role: str
    content: str

@dataclass
class ConversationManager:
    model: str
    summarize_every_k: int = 3
    system_prompt: str = "You are a concise helpful assistant."
    history: List[Message] = field(default_factory=list)
    turn_counter: int = 0

    def add_user(self, content: str) -> None:
        self.history.append(Message("user", content))

    def add_assistant(self, content: str) -> None:
        self.history.append(Message("assistant", content))

    def _messages_for_api(self) -> List[Dict[str, str]]:
        msgs = [{"role": "system", "content": self.system_prompt}]
        msgs.extend({"role": m.role, "content": m.content} for m in self.history)
        return msgs

    def chat(self, prompt: str) -> str:
        self.add_user(prompt)
        msgs = self._messages_for_api()
        if client:
            try:
                resp = client.chat.completions.create(
                    model=self.model,
                    messages=msgs,
                    temperature=0.2,
                )
            except Exception as e:
                # Try fallbacks if model is decommissioned or unavailable
                fallback_used = None
                for m in MODEL_FALLBACKS:
                    try:
                        resp = client.chat.completions.create(
                            model=m,
                            messages=msgs,
                            temperature=0.2,
                        )
                        fallback_used = m
                        break
                    except Exception:
                        continue
                if not fallback_used:
                    raise e
                else:
                    self.model = fallback_used
            output = resp.choices[0].message.content
        else:
            # Legacy fallback
            try:
                completion = openai.ChatCompletion.create(
                    model=self.model,
                    messages=msgs,
                    temperature=0.2,
                )
            except Exception as e:
                fallback_used = None
                for m in MODEL_FALLBACKS:
                    try:
                        completion = openai.ChatCompletion.create(
                            model=m,
                            messages=msgs,
                            temperature=0.2,
                        )
                        fallback_used = m
                        break
                    except Exception:
                        continue
                if not fallback_used:
                    raise e
                else:
                    self.model = fallback_used
            output = completion["choices"][0]["message"]["content"]
        self.add_assistant(output)
        self.turn_counter += 1
        if self.summarize_every_k > 0 and self.turn_counter % self.summarize_every_k == 0:
            self.periodic_summarize_replace()
        return output

    def truncate_last_n_turns(self, n: int) -> None:
        # One turn = user + assistant
        if n <= 0:
            self.history = []
            return
        to_keep = []
        turns = 0
        for m in reversed(self.history):
            to_keep.append(m)
            if m.role == "assistant":
                turns += 1
                if turns >= n:
                    break
        self.history = list(reversed(to_keep))

    def truncate_by_char_limit(self, max_chars: int) -> None:
        acc = 0
        to_keep = []
        for m in reversed(self.history):
            if acc + len(m.content) <= max_chars:
                to_keep.append(m)
                acc += len(m.content)
            else:
                break
        self.history = list(reversed(to_keep))

    def truncate_by_word_limit(self, max_words: int) -> None:
        acc = 0
        to_keep = []
        for m in reversed(self.history):
            words = len(m.content.split())
            if acc + words <= max_words:
                to_keep.append(m)
                acc += words
            else:
                break
        self.history = list(reversed(to_keep))

    def summarize_history(self, instruction: str = "Summarize the conversation so far in 3-5 bullet points.") -> str:
        if not self.history:
            return "(empty)"
        msgs = self._messages_for_api() + [{"role": "user", "content": instruction}]
        if client:
            resp = client.chat.completions.create(
                model=self.model,
                messages=msgs,
                temperature=0.0,
            )
            summary = resp.choices[0].message.content
        else:
            completion = openai.ChatCompletion.create(
                model=self.model,
                messages=msgs,
                temperature=0.0,
            )
            summary = completion["choices"][0]["message"]["content"]
        return summary

    def periodic_summarize_replace(self) -> None:
        summary = self.summarize_history(
            "Summarize the preceding conversation succinctly. Preserve key facts, intents, and constraints."
        )
        self.history = [Message("system", self.system_prompt), Message("assistant", f"[Summary]\n{summary}")]

cm = ConversationManager(model=GROQ_MODEL, summarize_every_k=3)
cm.add_assistant("Hi! I can help with your questions.")
print("Initialized conversation manager with periodic summarization every 3 turns.")


In [ ]:
# Demo: feed multiple conversation samples and show truncations

samples = [
    "What's the weather like in Paris?",
    "Can you also suggest 2 museums to visit?",
    "Book me a table for two at 7pm near Louvre.",
    "Change that to 8pm and make it for four people.",
]

for i, s in enumerate(samples, 1):
    print(f"\n--- Turn {i} ---")
    out = cm.chat(s)
    print("Assistant:", out[:200], ("..." if len(out) > 200 else ""))

print("\nHistory length after demo:", len(cm.history))

print("\nTruncate to last 2 turns (user+assistant pairs):")
cm.truncate_last_n_turns(2)
for m in cm.history:
    print(m.role + ":", m.content[:120])

print("\nTruncate by char limit (max 280 chars):")
cm.truncate_by_char_limit(280)
for m in cm.history:
    print(m.role + ":", m.content[:120])

print("\nTruncate by word limit (max 60 words):")
cm.truncate_by_word_limit(60)
for m in cm.history:
    print(m.role + ":", m.content[:120])



In [ ]:
# Periodic summarization demonstration

cm2 = ConversationManager(model=GROQ_MODEL, summarize_every_k=3)
cm2.add_assistant("Hello! I'm ready.")
seq = [
    "We discussed ordering pizza with extra cheese.",
    "Then we considered switching to pasta instead.",
    "Finally, confirm the pasta order for 2 people.",
    "Add garlic bread please.",
]

for i, s in enumerate(seq, 1):
    print(f"\n== Step {i} ==")
    out = cm2.chat(s)
    print("Assistant:", out[:180], ("..." if len(out) > 180 else ""))
    # Observe that after steps 3 and 6 etc., history will be replaced by summary
    print("History snapshot (roles):", [m.role for m in cm2.history])

print("\nFinal history after periodic summaries:")
for m in cm2.history:
    print(m.role + ":", m.content[:200])



In [ ]:
# Task 2: JSON Schema extraction via function calling

# Define a JSON schema-ish descriptor and a matching tool/function
extraction_schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "email": {"type": "string"},
        "phone": {"type": "string"},
        "location": {"type": "string"},
        "age": {"type": "integer"},
    },
    "required": ["name", "email", "phone", "location", "age"],
    "additionalProperties": False,
}

def get_tools_for_openai():
    # OpenAI-style tool/function spec
    return [
        {
            "type": "function",
            "function": {
                "name": "extract_contact_info",
                "description": "Extract structured contact details from a chat.",
                "parameters": extraction_schema,
            },
        }
    ]


def call_extraction(chat_text: str) -> Dict[str, Any]:
    messages = [
        {"role": "system", "content": "You extract structured data. If fields are missing, infer conservatively or leave blank."},
        {"role": "user", "content": f"From this chat, extract fields using the tool:\n\n{chat_text}"},
    ]

    models_to_try = [GROQ_MODEL] + [m for m in MODEL_FALLBACKS if m != GROQ_MODEL]

    if client:
        # First pass: auto tool choice
        last_err = None
        resp = None
        used_model = None
        for m in models_to_try:
            try:
                resp = client.chat.completions.create(
                    model=m,
                    messages=messages,
                    tools=get_tools_for_openai(),
                    tool_choice="auto",
                    temperature=0.0,
                )
                used_model = m
                break
            except Exception as e:
                last_err = e
                continue
        if resp is None:
            raise last_err

        choice = resp.choices[0]
        tool_calls = choice.message.tool_calls or []
        if tool_calls:
            args = json.loads(tool_calls[0].function.arguments)
            return args

        # Second pass: force the tool call
        resp2 = None
        for m in models_to_try:
            try:
                resp2 = client.chat.completions.create(
                    model=m,
                    messages=messages,
                    tools=get_tools_for_openai(),
                    tool_choice={"type": "function", "function": {"name": "extract_contact_info"}},
                    temperature=0.0,
                )
                used_model = m
                break
            except Exception:
                continue
        if resp2 is None:
            return {}
        choice2 = resp2.choices[0]
        tool_calls2 = choice2.message.tool_calls or []
        if tool_calls2:
            return json.loads(tool_calls2[0].function.arguments)
        return {}

    else:
        # Legacy path
        last_err = None
        completion = None
        for m in models_to_try:
            try:
                completion = openai.ChatCompletion.create(
                    model=m,
                    messages=messages,
                    tools=get_tools_for_openai(),
                    tool_choice="auto",
                    temperature=0.0,
                )
                break
            except Exception as e:
                last_err = e
                continue
        if completion is None:
            raise last_err

        choice = completion["choices"][0]
        tool_calls = choice["message"].get("tool_calls") or []
        if tool_calls:
            args = json.loads(tool_calls[0]["function"]["arguments"])
            return args

        # Force tool
        completion2 = None
        for m in models_to_try:
            try:
                completion2 = openai.ChatCompletion.create(
                    model=m,
                    messages=messages,
                    tools=get_tools_for_openai(),
                    tool_choice={"type": "function", "function": {"name": "extract_contact_info"}},
                    temperature=0.0,
                )
                break
            except Exception:
                continue
        if completion2 is None:
            return {}
        choice2 = completion2["choices"][0]
        tool_calls2 = choice2["message"].get("tool_calls") or []
        if tool_calls2:
            return json.loads(tool_calls2[0]["function"]["arguments"])
        return {}


def validate_against_schema(data: Dict[str, Any], schema: Dict[str, Any]) -> Dict[str, Any]:
    errors = []
    # Minimum manual validation without external libs
    if schema.get("type") == "object":
        props = schema.get("properties", {})
        required = schema.get("required", [])
        for field in required:
            if field not in data:
                errors.append(f"Missing required field: {field}")
        for k, v in data.items():
            if k not in props and not schema.get("additionalProperties", True):
                errors.append(f"Unexpected field: {k}")
        for k, prop in props.items():
            if k in data:
                expected_type = prop.get("type")
                val = data[k]
                if expected_type == "string" and not (isinstance(val, str) or val is None):
                    errors.append(f"Field {k} should be string")
                if expected_type == "integer" and not (isinstance(val, int) or (isinstance(val, str) and val.isdigit())):
                    errors.append(f"Field {k} should be integer")
    return {"ok": len(errors) == 0, "errors": errors}



In [ ]:
# Demo: parse at least 3 sample chats and validate

sample_chats = [
    """
    Hi I'm Alice Johnson. You can reach me at alice@example.com or +1-415-555-1212.
    I'm currently in San Francisco, and I'm 29 years old.
    """.strip(),
    """
    This is Bob, email bob.smith@workmail.io, phone 020 7946 0958.
    Based in London, age is 41.
    """.strip(),
    """
    Name: Carol Doe
    Email: carol.d@example.org
    Location: Toronto
    Phone: (647) 555-9988
    Age: 34
    """.strip(),
]

extracted = []
for i, chat in enumerate(sample_chats, 1):
    print(f"\n### Chat {i}")
    print(chat)
    data = call_extraction(chat)
    print("Extracted:", json.dumps(data, indent=2))
    result = validate_against_schema(data, extraction_schema)
    print("Validation:", result)
    extracted.append((data, result))

print("\nAll extractions complete.")


## Notes

- The conversation manager supports three truncation strategies and a configurable periodic summarization every k turns.
- The extraction uses OpenAI-style tools on Groq's API. A lightweight manual validator checks the schema rules without external libraries.
- Set `GROQ_API_KEY` and optionally `GROQ_MODEL` in the environment before running; otherwise you'll be prompted in Colab.
